In [ ]:
import pandas as pd
import json
with open(r"D:\shaha\Documents\python progams\AaveCreditScoring\user-wallet-transactions.json\user-wallet-transactions.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df.head())

                                    _id  \
0  {'$oid': '681d38fed63812d4655f571a'}   
1  {'$oid': '681aa70dd6df53021cc6f3c0'}   
2  {'$oid': '681d04c2d63812d4654c733e'}   
3  {'$oid': '681d133bd63812d46551b6ef'}   
4  {'$oid': '681899e4ba49fc91cf2f4454'}   

                                   userWallet  network protocol  \
0  0x00000000001accfa9cef68cf5371a23025b6d4b6  polygon  aave_v2   
1  0x000000000051d07a4fb3bd10121a343d85818da6  polygon  aave_v2   
2  0x000000000096026fb41fc39f9875d164bd82e2dc  polygon  aave_v2   
3  0x000000000096026fb41fc39f9875d164bd82e2dc  polygon  aave_v2   
4  0x0000000000e189dd664b9ab08a33c4839953852c  polygon  aave_v2   

                                              txHash  \
0  0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...   
1  0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...   
2  0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...   
3  0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...   
4  0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...   

 

In [10]:
df['wallet'] = df['userWallet']
df['action'] = df['action']
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df['amount'] = df['actionData'].apply(lambda x: float(x.get('amount', 0)))
df['price'] = df['actionData'].apply(lambda x: float(x.get('assetPriceUSD', 0)))
df['usd_value'] = df['amount'] * df['price']


In [11]:
features = df.groupby('wallet').agg(
    tx=('action', 'count'),
    deposits=('action', lambda x: (x == 'deposit').sum()),
    borrows=('action', lambda x: (x == 'borrow').sum()),
    repayments=('action', lambda x: (x == 'repay').sum()),
    liquidations=('action', lambda x: (x == 'liquidationcall').sum()),
    total_deposit_usd=('usd_value', lambda x: x[df['action'] == 'deposit'].sum()),
    total_borrow_usd=('usd_value', lambda x: x[df['action'] == 'borrow'].sum()),
    total_repay_usd=('usd_value', lambda x: x[df['action'] == 'repay'].sum()),
    first_tx_time=('timestamp', 'min'),
    last_tx_time=('timestamp', 'max'),
).reset_index()

# Time span in days
features['activity_days'] = (features['last_tx_time'] - features['first_tx_time']).dt.days + 1


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

# Drop non-numeric and nulls
model_features = features.select_dtypes(include=['number']).drop(columns=['activity_days'])
model_features = model_features.fillna(0)

# Model (unsupervised scoring simulation)
model = RandomForestRegressor(n_estimators=50, random_state=42)
model.fit(model_features, model_features.sum(axis=1))

# Score = model output normalized to 0–1000
raw_scores = model.predict(model_features)
scaled_scores = MinMaxScaler(feature_range=(0, 1000)).fit_transform(raw_scores.reshape(-1, 1)).flatten()
features['credit_score'] = scaled_scores.astype(int)
# Save the features with credit scores
features.to_csv('user_credit_scores.csv', index=False)